Rede neural CNN treinado sob dados brutos de forma binária

Importar as bibliotecas e os dados do conjunto MNIST

In [93]:
import json
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Normalizar e redimensionar os dados

In [94]:
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

Criar os filtros de classes

In [95]:
# Filtrar apenas duas classes (por exemplo, 0 e 1)
binary_classes = [0, 1]
train_filter = np.where((y_train == binary_classes[0]) | (y_train == binary_classes[1]))
test_filter = np.where((y_test == binary_classes[0]) | (y_test == binary_classes[1]))

Filtrar as classes

In [96]:
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

Construir o modelo da CNN

In [97]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

Compilar o modelo

In [98]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [99]:
# Definir os hiperparâmetros da arquitetura da rede neural e os hiperparâmetros de inicialização
hiperparametros = {
    'arquitetura': {
        'camadas': [
            {'tipo': 'Conv2D', 'filtros': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'input_shape': (28, 28, 1)},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'Flatten'},
            {'tipo': 'Dense', 'unidades': 64, 'activation': 'relu'},
            {'tipo': 'Dense', 'unidades': 1, 'activation': 'sigmoid'}
        ]
    },
    'inicializacao': {
        'optimizer': 'adam',
        'loss': 'binary_crossentropy',
        'metrics': ['accuracy']
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

Treinar o modelo

In [100]:
historico = model.fit(x_train, y_train, epochs=15, batch_size=64, validation_split=0.2)

Epoch 1/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9741 - loss: 0.1189 - val_accuracy: 0.9996 - val_loss: 0.0013
Epoch 2/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9991 - loss: 0.0037 - val_accuracy: 1.0000 - val_loss: 3.6327e-04
Epoch 3/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.9989 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 4.5898e-04
Epoch 4/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9998 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 1.9108e-04
Epoch 5/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9997 - loss: 9.1016e-04 - val_accuracy: 0.9996 - val_loss: 4.5359e-04
Epoch 6/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9994 - loss: 0.0011 - val_accuracy: 0.9984 - val_loss: 0.0051
Epoch 7/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9996 - loss: 7.6441e-04 - val_accuracy: 1.0000 - val_loss: 9.1731e-06
Epoch 8/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 1.00

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [101]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

396/396 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Testar o modelo

In [102]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 3.3362e-05
Test accuracy: 1.0, Test loss: 0.0002722359204199165
